<h1 align="center">Práctica 4. El problema del ensamblaje (Cota Voraz)</h1>
<h3 style="display:block; margin-top:5px;" align="center">Algorítmica</h3>
<h3 style="display:block; margin-top:5px;" align="center">Grado en Ingeniería Informática</h3>
<h3 style="display:block; margin-top:5px;" align="center">2024-2025</h3>    
<h3 style="display:block; margin-top:5px;" align="center">Universitat Politècnica de València</h3>
<br>

**Pon/poned aquí tú/vuestros nombre(s):**
- Yassin Pellicer Lamla

## Índice
1. ### [El problema del ensamblaje](#introduccion)
1. ### [Actividad 1: Solución voraz](#actividad1)

# El problema del ensamblaje

Se trata del problema descrito en los apuntes de teoría en la sección 9.4. Lo mejor es ir directamente al pdf para ver la descripción del problema, pero resumimos aquí algunos datos:

- Hay que ensamblar un total de $M$ piezas con el menor coste posible.
- El coste de ensamblar la pieza $i$ depende del número de piezas ya ensambladas.
- Los datos de entrada se resumen en una matriz `costes` de tamaño $M \times M$ con valores positivos (no hace falta que sean enteros). El valor `costes[i,j]` representa el coste de situar la pieza `i` (un identificador entre `0` y `M-1`) cuando ya se han ensamblado `j` piezas.
- Las soluciones son tuplas de la forma $(x_0,x_1,\ldots,x_{M-1})$ donde $x_i$ es el nº piezas ya montadas en el momento en que se decide montar la pieza que identificamos con el índice $i$.
- La función objetivo es: $f((x_0,x_1,\ldots,x_{M-1})) = \sum_{0 \leq i < M} costes [i,x_i]$
- Todas las permutaciones serían factibles, se trata de encontrar una que corresponda a un coste mínimo (podría haber empates).
- Se trata de un problema conocido en teoría de grafos, el [Problema de la asignación](https://es.wikipedia.org/wiki/Problema_de_la_asignaci%C3%B3n) o [Assignment problem](https://en.wikipedia.org/wiki/Assignment_problem) para el que existen algoritmos como [Kuhn Munkres](https://en.wikipedia.org/wiki/Hungarian_algorithm) con un coste polinómico ($O(|V|^3)$). Sería interesante comparar los algoritmos de ramificación y poda de esta práctica con estos otros, pero en una sola sesión no da tiempo.

## Generación de instancias

Para generar instancias concretas para una talla dada, vamos a recurrir a la generación de números aleatorios utilizando la siguiente función de la biblioteca `numpy`:

In [1]:
import numpy as np
def genera_instancia(M, low=1, high=1000):
    return np.random.randint(low=low,high=high,
                             size=(M,M),dtype=np.int32)

In [4]:
costes = genera_instancia(4,high=10)
costes

array([[5, 7, 8, 4],
       [3, 4, 7, 7],
       [2, 6, 9, 3],
       [9, 5, 4, 4]], dtype=int32)

Con una matriz como ésta (cada vez que lo ejecutes dará normalmente otra distinta):

```python
array([[7, 3, 7, 2],
       [9, 9, 4, 1],
       [9, 4, 8, 1],
       [3, 4, 8, 4]])
```

el coste de ensamblar la pieza 0 en la cuarta posición (después de haber ensamblado 3 piezas) es `costes[0,3]` que vale `2`.


## Representación de los estados

- El estado inicial será la lista vacía `[]`.
- Un estado intermedio $(x_0,x_1,\ldots,x_{k-1})$ se representará mediante una lista Python con esos mismos valores.
- Un estado solución será una lista de talla $M$.

<a id='actividad1'></a>

# Actividad 1: Solución voraz

Se trata de obtener una solución con la que inicializar la variable `x` (mejor solución encontrada hasta el momento) y su score correspondiente `fx`.

En el problema del ensamblaje es trivial obtener una solución porque cualquier permutación de índices entre `0` y `M-1` es una solución válida. Es decir, podríamos hacer algo así:

```python
def ensamblaje(costes,
               verbosity=1):

    ...
    
    def naive_solution():
        score, solution = 0, []
        for i in range(M):
            solution.append(i)
            score += costes[i,i]
        return score,solution
```

Es decir, cada $x_i$ es igual a $i$ (los objetos se ponen en orden 0,1,2,...).

Pero cuanto mejor sea la solución inicial antes empezaremos a podar mejor, sin llegar al extremo de calcularla de forma exacta porque, en ese caso ¿para qué usar luego ramificación y poda?

En esta primera actividad debes completar la función `greedy_solution` donde existen varias opciones, entre ellas:

1. Ir por orden pieza a pieza (fila $i$ de la matriz) y elegir el valor $x_i$ (el momento de colocación de esa pieza) que resulte más barato de los que siguen disponibles. Es decir, ir fila por fila de la matriz y elegir (para esa fila) la columna menor de las columnas previamente no elegidas.
2. Ir por orden instante a instante (columna de la matriz) y elegir para cada una la pieza (fila de la matriz) que sea más barata de colocar en ese instante (de entre las piezas que queden por ensamblar).
3. Ordenar de menor a mayor todos los valores de la matriz de costes recordando sus coordenadas. Después se recorre utilizando los valores que correspondan a piezas e instantes válidos (descartando el resto) hasta haber situado todas las piezas.
4. Cualquier combinación de los anteriores (se calculan soluciones con varios algoritmos y nos quedamos con la mejor solución).

Debes implementar al menos la primera, las otras dos se pueden implementar de forma opcional.

In [87]:
"""
    Algoritmo voraz para inicializar a una "solución cualquiera"
    NO BUSCAMOS LA MEJOR SOLUCIÓN, solo una razonable que calcule en un tiempo razonable
"""
from numpy import sort

M = 4

def naive_solution():
    score, solution = 0, []
    for i in range(M):
        solution.append(i)
        score += costes[i,i]
    return int(score), solution

def greedy_solution1():
    """
    En este caso vamos fila por fila y elegimos la columna de
    menor valor de entre las que queden disponibles.
    """
    score, solution, chosen = 0, [None]*M, set()
    for i in range(M):
        subset = [j for j in range(M) if j not in chosen]
        chosen.add(subset[np.argmin(costes[i,subset])])
        solution[i] = subset[np.argmin(costes[i,subset])]
        score = score + costes[i,subset[np.argmin(costes[i,subset])]]
    return int(score), solution
    
def greedy_solution2():
    """
    Algoritmo voraz para inicializar a una "solución cualquiera"
    (la mejor que se pueda en un tiempo razonable)

    En este caso vamos columna por columna y elegimos la fila de
    menor valor de entre las que queden disponibles.
    """
    score, solution, chosen = 0, [None]*M, set()
    for i in range(M):
        subset = [j for j in range(M) if j not in chosen]
        chosen.add(subset[np.argmin(costes[subset, i])])
        solution[i] = subset[np.argmin(costes[subset, i])]
        score = score + costes[subset[np.argmin(costes[subset, i])],i]
    return int(score), solution

def greedy_solution3():
    """
    Ordenar de menor a mayor todos los valores de la matriz de costes recordando sus coordenadas.
    Después se recorre utilizando los valores que correspondan a piezas e instantes válidos.
    """
    M = costes.shape[0] # nº piezas

    # COMPLETAR

    # Inicializar la solución con -1
    solution = [-1] * M
    solution_aux = [-1] * M
    filas = [-1] * M
    cols = [-1] * M
    min=100000
    # Ordenar las tareas por coste
    #sorted_tasks = sorted(range(M), key=lambda i: costes[i,0])
    # Iterar a través de las tareas ordenadas por coste
    for k in range(M):
        for i in range(M):
            for j in range(M):
                # Encuentra el intervalo de tiempo con el menor coste para la tarea actual
                if(costes[i,j]<min and i not in filas and j not in cols):
                    min=costes[i,j]
                    col=j
                    fila=i
        min=10000
        filas[k]=fila
        cols[k]=col 
        # Asignar la tarea actual al intervalo de tiempo con el menor coste
        solution_aux[k]=(fila,col)
    solution_aux=sorted(solution_aux, key=lambda i: i[0])
    for i in range(M):
        solution[i]=solution_aux[i][1]
    # Calcular el coste total de la solución
    return ((sum(int(costes[pieza,instante])
        for pieza,instante in enumerate(solution)), solution))
   
def greedy_solution4():
    """
    La mejor de las tres
    """
    return sorted([greedy_solution1(), greedy_solution2(), greedy_solution3()])[0]

def run_greedy():
    print(costes)
    print('nv', naive_solution())
    print('g1' , greedy_solution1())
    print('g2' , greedy_solution2())
    print('g3', greedy_solution3())

## Prueba con una matriz conocida

In [88]:
costes = np.array(
    [
        [7, 3, 7, 2],
        [9, 9, 4, 1],
        [9, 4, 8, 1],
        [3, 4, 8, 4]
    ])
run_greedy()

[[7 3 7 2]
 [9 9 4 1]
 [9 4 8 1]
 [3 4 8 4]]
nv (28, [0, 1, 2, 3])
g1 (13, [3, 2, 1, 0])
g2 (11, [3, 0, 1, 2])
g3 (15, [1, 3, 2, 0])


La salida debe ser:
```python
[[7 3 7 2]
 [9 9 4 1]
 [9 4 8 1]
 [3 4 8 4]]
nv (28, [0, 1, 2, 3])
g1 (13, [3, 2, 1, 0])
g2 (11, [1, 2, 3, 0])
g3 (15, [1, 3, 2, 0])
```

In [89]:
M = 4
costes = np.array([[4, 4, 5, 3],
                   [2, 8, 9, 1],
                   [6, 9, 6, 3],
                   [4, 6, 7, 7]],dtype=np.int32)
run_greedy()

[[4 4 5 3]
 [2 8 9 1]
 [6 9 6 3]
 [4 6 7 7]]
nv (25, [0, 1, 2, 3])
g1 (17, [3, 0, 2, 1])
g2 (19, [1, 0, 2, 3])
g3 (17, [0, 3, 2, 1])


La salida debe ser:
```python
[[4 4 5 3]
 [2 8 9 1]
 [6 9 6 3]
 [4 6 7 7]]
nv (25, [0, 1, 2, 3])
g1 (17, [3, 0, 2, 1])
g2 (19, [1, 0, 2, 3])
g3 (17, [0, 3, 2, 1])
```

In [90]:
M = 10
costes = np.array([[30, 42, 65, 12, 45, 30, 99, 93,  1, 41],
       [88, 96, 72, 37, 14, 67, 28, 85, 46, 50],
       [78, 89, 86, 69, 48,  6, 79, 34, 85, 66],
       [88, 51, 15, 92, 72, 82, 36, 98, 97, 72],
       [ 5, 55, 58, 20, 56,  3, 97, 91, 99, 69],
       [18, 13, 99, 92, 39, 49, 29, 82, 47, 32],
       [35,  7, 58,  1, 47, 22, 80, 65, 51, 35],
       [42, 72, 31, 11, 88, 17, 81, 76,  4,  7],
       [33, 83,  6, 97, 24, 92, 94, 57, 86, 31],
       [56, 55, 41, 24, 65, 56, 73, 70, 86, 66]], dtype=np.int32)
run_greedy()

[[30 42 65 12 45 30 99 93  1 41]
 [88 96 72 37 14 67 28 85 46 50]
 [78 89 86 69 48  6 79 34 85 66]
 [88 51 15 92 72 82 36 98 97 72]
 [ 5 55 58 20 56  3 97 91 99 69]
 [18 13 99 92 39 49 29 82 47 32]
 [35  7 58  1 47 22 80 65 51 35]
 [42 72 31 11 88 17 81 76  4  7]
 [33 83  6 97 24 92 94 57 86 31]
 [56 55 41 24 65 56 73 70 86 66]]
nv (717, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
g1 (192, [8, 4, 5, 2, 0, 1, 3, 9, 7, 6])
g2 (221, [4, 6, 8, 7, 1, 2, 5, 9, 0, 3])
g3 (171, [8, 4, 7, 6, 5, 1, 3, 9, 2, 0])


La salida debe ser:
```python
[[30 42 65 12 45 30 99 93  1 41]
 [88 96 72 37 14 67 28 85 46 50]
 [78 89 86 69 48  6 79 34 85 66]
 [88 51 15 92 72 82 36 98 97 72]
 [ 5 55 58 20 56  3 97 91 99 69]
 [18 13 99 92 39 49 29 82 47 32]
 [35  7 58  1 47 22 80 65 51 35]
 [42 72 31 11 88 17 81 76  4  7]
 [33 83  6 97 24 92 94 57 86 31]
 [56 55 41 24 65 56 73 70 86 66]]
nv (717, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
g1 (192, [8, 4, 5, 2, 0, 1, 3, 9, 7, 6])
g2 (221, [8, 4, 5, 9, 0, 6, 1, 3, 2, 7])
g3 (171, [8, 4, 7, 6, 5, 1, 3, 9, 2, 0])
```

## Prueba con el generador de instancias

In [91]:
M = 5
for i in range(5):
    costes = genera_instancia(M, low=1, high=100)
    run_greedy()
    print('\n')

[[ 2 33 99 96 48]
 [39 59 21 66 44]
 [40 26 16 13 67]
 [86 30 96 66 17]
 [14 75 64 59 98]]
nv (241, [0, 1, 2, 3, 4])
g1 (128, [0, 2, 3, 4, 1])
g2 (125, [0, 2, 1, 4, 3])
g3 (128, [0, 2, 3, 4, 1])


[[ 9 70  7 57 96]
 [94 92 31 93 89]
 [72 25 17 91 17]
 [64 31 63  2 15]
 [ 5 66 14 32 91]]
nv (211, [0, 1, 2, 3, 4])
g1 (128, [2, 4, 1, 3, 0])
g2 (128, [4, 2, 0, 3, 1])
g3 (123, [2, 1, 4, 3, 0])


[[48 78  9 18 46]
 [68 15 61  9 99]
 [99 19 65 52 90]
 [92 91 15 56  1]
 [94  5 49 42 73]]
nv (257, [0, 1, 2, 3, 4])
g1 (132, [2, 3, 1, 4, 0])
g2 (167, [0, 4, 3, 1, 2])
g3 (123, [2, 3, 0, 4, 1])


[[32  9  4 46  9]
 [89 88 11 99 96]
 [33 72  3 95 75]
 [61 99 45 44 44]
 [82  2 13 11 86]]
nv (253, [0, 1, 2, 3, 4])
g1 (255, [2, 1, 0, 3, 4])
g2 (177, [0, 4, 2, 3, 1])
g3 (147, [4, 0, 2, 3, 1])


[[58  1 79 31 95]
 [86 37 23  3 24]
 [19 60 78 56 50]
 [45 89 96 85 87]
 [40  4 86 76  5]]
nv (263, [0, 1, 2, 3, 4])
g1 (196, [1, 3, 0, 4, 2])
g2 (206, [2, 0, 1, 4, 3])
g3 (124, [1, 3, 0, 2, 4])




# NOTA: no olvides copiar las soluciones voraces en el código de la siguiente práctica (Actividad 3 de la práctica completa)

<a id='actividad3'></a>

<a id='codigo'></a>